In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
import seaborn as sns

In [ ]:
def get_merged_csv(flist, **kwargs):
    return pd.concat([pd.read_csv(f, **kwargs) for f in flist], ignore_index=True)

path = os.getcwd() + "/data"
fmask = os.path.join(path, '*기보고서*.txt')
glob.glob(fmask)
df = get_merged_csv(glob.glob(fmask), index_col=None, sep = "\t", encoding = "cp949")

In [ ]:
# '재무제표종류 종목코드 회사명 시장구분 업종 업종명 결산월 결산기준일 
# 보고서종류 통화 항목코드 항목명 당기 1분기말 전기말 전전기말 Unnamed: 15 당기 3분기말 당기 반기말'

# 변수 삭제
" ".join(df.columns)

lst = ["통화", "Unnamed: 15"]
df.drop(lst, axis = 1, inplace = True)

In [ ]:
df.info()

In [ ]:
# 1분기보고서 : 결산월 3월 이면 결산기준일 6/30
# 결산월 부터 
df[df["보고서종류"] == "3분기보고서"][["결산기준일", "결산월"]]

In [ ]:
a = df[["업종명", "업종"]].groupby(["업종명"]).agg(["count"]).reset_index().sort_values(by = ( '업종', 'count'))
a.to_csv("./data/업종명.csv", encoding = "utf-8", index = False)

In [ ]:
b = a = df[["업종명", "업종"]].groupby(["업종"]).agg(["count"]).reset_index().sort_values(by = ( '업종명', 'count'))
b.to_csv("./data/업종.csv", encoding = "utf-8", index = False)

In [ ]:
print(a)
print(len(df["업종명"].unique()))
print(len(df["업종"].unique()))

In [ ]:
# 업종, 업종명 변수 확인 및 처리

lst = df["업종명"].unique()

check = df[["업종명", "업종"]]

lst2 = []
for i in lst:
    lst2.append(check[check["업종명"] == i].iloc[0,1])
    
check_df1 = pd.DataFrame({"업종명1":lst, "업종":lst2})
# check_df.to_csv("업종명기준확인.csv", encoding = "utf-8", index = False)

lst = df["업종"].unique()

check = df[["업종", "업종명"]]

lst2 = []
for i in lst:
    lst2.append(check[check["업종"] == i].iloc[0,1])
    
check_df2 = pd.DataFrame({"업종":lst, "업종명2":lst2})
pd.merge(check_df1, check_df2, how = "outer").to_csv("최종확인.csv", encoding = "utf-8", index = False)

In [ ]:
# 2019년도 결산기준일 부터는 업종명이 통일 되어 있으므로
# 19년도 행을 기반으로 업종명 추출
check_idx = []
for i in range(len(df)):
    if df["결산기준일"][i][:4] == "2019":
        check_idx.append(i)
        
check = df.iloc[check_idx,:].reset_index(drop=True)
lst_업종 = list(check["업종"].unique())
lst_업종명 = []

for i in lst_업종:
    lst_업종명.append(check[check["업종"] == i].iloc[0, 5])
    
# 19년도에 없는 업종과 업종명 추가
# index순서가 중요하다(why? 업종의 인덱스 위치와 업종명의 인덱스 위치가 같아야 되기 때문)
lst_업종.append(693)
lst_업종명.append("산업용 기계 및 장비 임대업")
lst_업종.append(691)
lst_업종명.append("운송장비 임대업")
lst_업종.append(71)
lst_업종명.append("토사석 광업")
lst_업종.append(302)
lst_업종명.append("자동차 차체 및 트레일러 제조업")
lst_업종.append(12)
lst_업종명.append("축산업")

# 업종 id, 업종명 value
df["업종명"] = None
dic = {i:j for i, j in zip(lst_업종, lst_업종명)}

# 19년도 기준 업종명 추가 
for i in range(len(df)):
    df["업종명"][i] = dic[df["업종"][i]]

In [ ]:
for i in range(len(df)):
    df["업종명"][i] = dic[df["업종"][i]]